Revenue growth rate

In [72]:
# Inputs
rev_gr_1 = 12/100  # Revenue growth rate for year 1
cagr_2_5 = 12/100   # CAGR for years 2-5
risk_free_rate = 4.08/100 # Risk free rate
rev_r12m = 574785 # Revenue for the last 12 months (in millions)
ebit_r12m = 36852 # EBIT for the last 12 months (in millions)
rnd_amr_yrs = 3 # R&D amortization years
curr_rnd = 85622 # Current R&D (in millions)
prev_rnd = [73213, 56052, 42740, 0, 0, 0, 0, 0, 0, 0] # Previous R&D (in millions)
#ebit_margin_1 = 11.33/100 # Operating margin for year 1 
tgt_ebit_margin = 14/100 # Target Pre-tax Operating margin
yr_conv_margin = 5 # Years to convert to target margin

Growth rates list

In [34]:
# Create a list of 10 growth rates
growth_rates = [rev_gr_1]  # First year growth rate

# Year two to five will be cagr_2_5
growth_rates.extend([cagr_2_5] * 4)

# Calculate the decrement for years six to ten
decrement = (cagr_2_5 - risk_free_rate) / 5

# Year six to ten growth rates
for i in range(1, 6):
    growth_rates.append(cagr_2_5 - decrement * i)

print(growth_rates)

[0.12, 0.12, 0.12, 0.12, 0.12, 0.10416, 0.08832, 0.07247999999999999, 0.056639999999999996, 0.0408]


Revenues list

In [35]:
revenues = [rev_r12m * (1 + growth_rates[0])]  # Revenue for year 1

# Calculate revenues for subsequent years
for i in range(1, len(growth_rates)):
    revenues.append(revenues[-1] * (1 + growth_rates[i]))

print(revenues) 

[643759.2000000001, 721010.3040000001, 807531.5404800002, 904435.3253376003, 1012967.5643781124, 1118478.2658837365, 1217262.266326588, 1305489.4353899392, 1379432.3570104255, 1435713.1971764509]


Amortization weights

In [36]:
# Calculate the number of non-zero entries in prev_rnd
non_zero_entries = sum(1 for rnd in prev_rnd if rnd != 0)

# Calculate the amortization weights
amortization_weights = [i / non_zero_entries for i in range(non_zero_entries, 0, -1)]

print(amortization_weights)

[1.0, 0.6666666666666666, 0.3333333333333333]


In [37]:
# Value of research asset
val_rnd = sum(a * b for a, b in zip(([curr_rnd] + prev_rnd)[:len(amortization_weights)], amortization_weights))
val_rnd

153114.66666666666

In [38]:
#Amortization for this year
sum_prev_rnd = sum([rnd / rnd_amr_yrs for rnd in prev_rnd])
sum_prev_rnd

57335.0

In [39]:
# Adjustment to EBIT due to R&D
adj_ebit = curr_rnd - sum_prev_rnd
adj_ebit

28287.0

Adjusted operating income

In [41]:
# Adjusted EBIT (Base Year)
if curr_rnd != 0:
    ebit_adj = ebit_r12m + adj_ebit
else: 
    ebit_adj = ebit_r12m
ebit_adj

65139.0

Operating Margins

In [68]:
ebit_margin_0 = ebit_adj / rev_r12m
ebit_margin_1 = ebit_margin_0 # Assuming same margin for year 1

In [69]:
# Initialize the list with the operating margin for the first year
operating_margins = [ebit_margin_1]

# Calculate the step change for the operating margin
step_change = (tgt_ebit_margin - ebit_margin_1) / (yr_conv_margin)

# Calculate the operating margins for years 2 to the year of convergence
for i in range(1, yr_conv_margin):
    operating_margins.append(tgt_ebit_margin - step_change*(yr_conv_margin - i - 1))

# From the year of convergence onwards, the operating margin remains the same as the target one
operating_margins.extend([tgt_ebit_margin] * (10 - yr_conv_margin + 1))

print(operating_margins)

[0.11332759205616012, 0.12399655523369607, 0.12933103682246405, 0.13466551841123203, 0.14, 0.14, 0.14, 0.14, 0.14, 0.14, 0.14]


Operating Income

In [71]:
ebits = [revenue * margin for revenue, margin in zip(revenues, operating_margins)]
print(ebits)

[72955.68000000001, 89402.793984, 104438.89139712002, 121796.25195601924, 141815.45901293575, 156586.95722372312, 170416.71728572235, 182768.5209545915, 193120.5299814596, 200999.84760470313]
